In [4]:
import pandas as pd
import tabula
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import random
from medscraper import medsearch

In [5]:
#Display solo 10 filas
pd.set_option('display.max_rows', 10)

In [7]:
#Archivo DGIL
dgil=pd.read_excel('Lista de precios de medicamentos_Diciembre_2022.xlsx',skiprows=(4), usecols=(range(5)))
dgil = dgil.iloc[:-2]
dgil.columns=dgil.columns.str.lower()
#Renmbrar penúltima y última columna
fourth_column_name = dgil.columns[3]
fifth_column_name = dgil.columns[4]
dgil.rename(columns={fourth_column_name:'pres',fifth_column_name:'precio'}, inplace=True)
#Eliminar filas vacías
dgil=dgil.dropna()
#Eliminar espacios en blanco
dgil['producto']=dgil['producto'].str.strip()
dgil

FileNotFoundError: [Errno 2] No such file or directory: 'Lista de precios de medicamentos_Diciembre_2022.xlsx'

In [6]:
#Eliminar duplicados
dgil=dgil.drop_duplicates(subset=['producto'], keep='first')

NameError: name 'dgil' is not defined

In [4]:
dgil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 0 to 380
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empresa           353 non-null    object 
 1   producto          353 non-null    object 
 2   principio activo  353 non-null    object 
 3   pres              353 non-null    object 
 4   precio            353 non-null    float64
dtypes: float64(1), object(4)
memory usage: 16.5+ KB


In [5]:
#Leer datos de cofepris
#url="https://www.gob.mx/cms/uploads/attachment/file/803597/LMR_2023-01_actualizaci_n_10_febrero_2023.pdf"


In [6]:
#cofepris=tabula.read_pdf(url,pages='all',multiple_tables=True,lattice=True)

In [7]:
# cofepris_df=pd.DataFrame()
# for tablas in cofepris:
#     cofepris_df=pd.concat([cofepris_df,tablas],ignore_index=True)

# #Renombrar columna 1
# cofepris_df.rename(columns={cofepris_df.columns[0]:'consecutivo',cofepris_df.columns[1]:"farmaco",
#                             cofepris_df.columns[2]: "forma", cofepris_df.columns[3]:"concentra",
#                             cofepris_df.columns[4]:"fraccion",cofepris_df.columns[5]:
#                             "producto",cofepris_df.columns[6]:"registro",
#                             cofepris_df.columns[7]:"titular",
#                             cofepris_df.columns[8]:"terapeutica",
#                             },inplace=True)
# #Eliminar vacios de columna "consecutivo"
# cofepris_df=cofepris_df.dropna(subset=['consecutivo'])
# #Resetear index
# cofepris_df=cofepris_df.reset_index(drop=True)  
# #Eliminar espacios en blanco en columna "producto"
# cofepris_df['producto']=cofepris_df['producto'].str.strip()

In [8]:
cofepris_df=pd.read_excel("cofepris.xlsx")

In [9]:
cofepris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   consecutivo  1526 non-null   int64 
 1   farmaco      1526 non-null   object
 2   forma        1526 non-null   object
 3   concentra    1526 non-null   object
 4   fraccion     1526 non-null   object
 5   producto     1526 non-null   object
 6   registro     1526 non-null   object
 7   titular      1526 non-null   object
 8   terapeutica  1526 non-null   object
dtypes: int64(1), object(8)
memory usage: 107.4+ KB


In [10]:
cofepris_df

,consecutivo,farmaco,forma,concentra,fraccion,producto,registro,titular,terapeutica
0,1,"ABACAVIR, Sulfato de",Tableta,300 mg,IV,ZIAGENAVIR,088M99 SSA,"GLAXO WELLCOME, S.A.",Antirretroviral.
1,2,"ABACAVIR, Sulfato de",Jarabe,2 g / 100 mL,IV,ZIAGENAVIR,087M99 SSA,"GLAXO WELLCOME, S.A.",Antirretroviral.
2,3,"ABACAVIR, Sulfato de /\rLAMIVUDINA",Tableta,600 mg / 300 mg,IV,KIVEXA,479M2005 SSA,"GLAXO WELLCOME, S.A.",Antirretroviral. Para el tratamiento de la inf...
3,4,"ABIRATERONA, Acetato de",Tableta,250 mg y 500 mg,IV,ZYTIGA,241M2012 SSA,"JANSSEN-CILAG, S.A. DE C.V.",En el tratamiento de pacientes con cáncer de p...
4,5,ACARBOSA,Tableta,50 mg y 100 mg,IV,GLUCOBAY,458M93 SSA,"BAYER DE MÉXICO, S.A. DE C.V.",Para el tratamiento de la diabetes mellitus ti...
...,...,...,...,...,...,...,...,...,...
1521,1522,ZOLPIDEM,Tableta,10mg,III,STILNOX,044M98 SSA,"SANOFI-AVENTIS DE MÉXICO,\rS.A. DE C.V.",Insomnio.
1522,1523,"ZOLPIDEM, Tartrato de",Tableta\r(Liberación prolongada),6.25 mg y 12.5 mg,III,STILNOX CR,152M2008 SSA,"SANOFI-AVENTIS DE MÉXICO,\rS.A. DE C.V.",I n d u c t o r d e l s u eño (pa...
1523,1524,"ZUCLOPENTIXOL, Acetato de",Solución\r(Inyectable),50 mg/1 mL,IV,CLOPIXOL ACUPHASE,414M96 SSA,H. LUNDBECK A/S,Antipsicótico.
1524,1525,"ZUCLOPENTIXOL, Decanoato de",Solución\r(Inyectable),200 mg/1 mL,IV,CLOPIXOL DEPOT,415M96 SSA,H. LUNDBECK A/S,Antipsicótico. Tratamiento inicial de la esqui...


In [11]:
#Cruce de ambos dataframes
cruce=pd.merge(dgil,cofepris_df,how='left',on='producto')
cruce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 0 to 358
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   empresa           359 non-null    object 
 1   producto          359 non-null    object 
 2   principio activo  359 non-null    object 
 3   pres              359 non-null    object 
 4   precio            359 non-null    float64
 5   consecutivo       124 non-null    float64
 6   farmaco           124 non-null    object 
 7   forma             124 non-null    object 
 8   concentra         124 non-null    object 
 9   fraccion          124 non-null    object 
 10  registro          124 non-null    object 
 11  titular           124 non-null    object 
 12  terapeutica       124 non-null    object 
dtypes: float64(2), object(11)
memory usage: 39.3+ KB


In [12]:
#Contar valores nulos de registro
cruce['registro'].isnull().sum()

235

## Segundo método: buscar en el catálogo de COFEPRIS

In [13]:
def busca_cofepris(medicamento):
    #función para cambiar la denominación del medicamento
    def denominacion(option_text):
        combo = driver.find_element(By.CLASS_NAME, "form-control")
        combo_select = Select(combo)
        combo_select.select_by_visible_text(option_text)
        cuadro_busqueda = driver.find_element(By.ID, "MainContent_txtSearchValue")
        cuadro_busqueda.send_keys(medicamento)
        button = driver.find_element(By.XPATH, "//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
        button.click()
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
        return table_contents
    
    #Código para extraer los datos de la página
    url = "https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"), options=options)
    driver.get(url)
    #Denominación Distintiva
    denominacion("Denominación Distintiva")

    try:
        table_contents = elementos_tabla(driver)
    except NoSuchElementException:
        #Denominación Genérica
        denominacion("Denominación Genérica")
        try:
            table_contents = elementos_tabla(driver)
        except NoSuchElementException:
            driver.quit()
            return pd.DataFrame()

    soup = BeautifulSoup(table_contents, 'html.parser')
    registro=[]
    marca = []
    generica = []
    indica = []
    lab = []
    fabricante = []
    principio=[]

    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblNumeroRegistro'):
                registro.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblPrincipioActivo'):
                principio.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblIndicacionTerapeutica'):
                indica.append(span.text)

    df = pd.DataFrame({'reg_san':registro,'marca': marca, 'den_gen': generica, 'titular': lab, 'principio': principio,'indicacion':indica,"busqueda":medicamento})
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    driver.quit()
    return df


In [14]:
muestra=dgil.sample(3)

In [15]:
muestra

,empresa,producto,principio activo,pres,precio
271,"Novartis Farmacéutica, S.A. de C.V.",TASIGNA,Nilotinib,Caja con 120 tabletas de 150 mg,56511.00
314,"Productos Roche, S.A. de C.V.",MIRCERA,Eritropoyetina beta,Caja con 1 jeringa prellenada de 250 mg / 0.3 ml.,10700.00
181,"Janssen-Cilag de México, S. de R.L. de C.V.",SIMPONI,Golimumab,Pluma precargada con 50mg por 0.5 ml,26904.43


In [16]:
#Buscar medicamentos en cofepris
busqueda=pd.DataFrame()
for i in dgil['producto']:
    time.sleep(random.randint(1,3))
    busqueda=pd.concat([busqueda,busca_cofepris(i)],ignore_index=True)

In [17]:
busqueda

,reg_san,marca,den_gen,titular,principio,indicacion,busqueda
0,163300el870091,venclexta,venetoclax,,l01xx52 venetoclax,tratamiento como monoterapia o en combinación ...,venclexta
1,163300el870091,venclexta,venetoclax,,l01xx52 venetoclax,tratamiento como monoterapia o en combinación ...,venclexta
2,392m2017 ssa,mavyret,glecaprevir / pibrentasvir,,,tratamiento de la infección crónica por el vir...,mavyret
3,013m2021 ssa,rim-voq,upadacitinib,,l04aa99 otros principios activos. agentes in...,para el tratamiento de adultos con artritis re...,rim-voq
4,144m2021 ssa,skyrizi,risankizumab,,l04ac18 risankizumab,psoriasis en placasskyrizi está indicado para ...,skyrizi
...,...,...,...,...,...,...,...
368,441m2016 ssa,briviact,brivaracetam,,n03ax23 brivaracetam,monoterapia y terapia de adición en el tratami...,briviact
369,386m2016 ssa,briviact,brivaracetam,,n03ax23 brivaracetam,indicada como monoterapia y terapia adjunta en...,briviact
370,379m2016 ssa,briviact,brivaracetam,,n03ax23 brivaracetam,monoterapia y terapia de adición en el tratami...,briviact
371,441m2016 ssa,briviact,brivaracetam,,n03ax23 brivaracetam,monoterapia y terapia de adición en el tratami...,briviact


In [18]:
#Contar valores únicos de búsqueda
busqueda['busqueda'].nunique()

132

In [25]:
busqueda['busqueda'].nunique()/dgil["producto"].nunique()*100

84.07643312101911

In [23]:
busqueda.to_excel("cruce_dgil_cofepris.xlsx")